In [ ]:
import os
import json

import pandas as pd

In [ ]:
# Set paths
ECHR_DATA_FILE = '../data/ECHR_metadata_0till600.json'
NETWORK_FILE = '../data/network.json'

In [ ]:
# Load JSON file
with open(ECHR_DATA_FILE, 'r', encoding='utf-8') as f:
    data = json.load(f) 
data = data['results']

# Extract columns and keys from result nested json
list_of_dicts = [k.get('columns') for k in data]

# Filter relevant columns
df_citation_check = pd.DataFrame(list_of_dicts)
df_citation_check = df_citation_check[['itemid','languageisocode','appno','extractedappno','scl']]

df_citation_check

In [ ]:
# Load JSON file
with open(NETWORK_FILE, 'r', encoding='utf-8') as f:
    df_to_merge = pd.read_json(f)
# Pre arrange df
df_to_merge = df_to_merge.transpose()
df_to_merge['itemid'] = df_to_merge.index

# Filter relevant columns
df_to_merge = df_to_merge[['itemid','references']]
df_to_merge

In [ ]:
mergedDf = pd.merge(df_citation_check,df_to_merge, on='itemid')
mergedDf

In [ ]:
#TO DO
"""Goal is to enrich the extracted citations by HUDOC with parsed citations with the method explained in https://github.com/jorgecarleitao/echr_network/. 
The necessity follows from the issue that HUDOC harvests citations, but not all, especially the ones where no application number is mentioned, 
whereas non-HUDOC scripts do not capture many of the HUDOC-extracted citations.

Preprocessing
1. Remove application numbers in exractedappno that are the same as the application number in appno (eg extractedappno=(38042/06;30979/96;31932/03) and appno=38042/06) 

Explore how frequently citations are missing when comparing the two columns in the HUDOC-metadata 
2. Create column that calculates whether the number of citations in the scl column exceeds the number of citations in the extractedappno column or v.v.
Do this for the complete df ['citation_diff_extr_vs_scl_all'] and the df with only English documents ['citation_diff_extr_vs_scl_ENG']:
- If number of citations in extractedappno > number of citations in scl: score=1
- If number of citations in extractedappno < number of citations in scl: score=-1
- If number of citations in extractedappno = number of citations in scl: score=0 

Insert missing citations in HUDOC-based dataset
3. For each application number (appno), merge the application numbers from the extractedappno (for the different languages) column.

Explore how frequently citations are missing in the HUDOC-metadata and/or in the parsed data
4. Create column that calculates whether the number of citations in the extracteappno column exceeds the number of citations in the references column based on the parsed references (and v.v.)
- If number of citations in extractedappno > number of citations in references: score=1
- If number of citations in extractedappno < number of citations in references: score=-1
- If number of citations in extractedappno = number of citations in references: score=0 

Update the extractedappno column where the parsed references include additional citations
6. Create column that copies references column. Replace itemids in references with application numbers
7. Create column named extractedappno_enhanced. Copy the values of extractedappno in there
8. If reference in references not in extractedappno_enhaned:
    - Add reference to extractedappno_enhanced.
    - Else: pass
[TO DO: manually check for samples (eg 100 where score=1, 100 where score=-1, 100 where score=0) whether scl contains decisions that are not mentioned in extractedappno and v.v. Report results]
9. Create new column ['extractedecli'], where the application numbers from extractedappno are replaced with the corresponding eclis
10. Export citation network to csv file. The network should look like this (ECLIs are abbreviated below):
    Source,Target
    ECLI:1,ECLI:2
    ECLI:1,ECLI:3
    ECLI:2,ECLI:3
    ECLI:3,ECLI:45
    ...
"""